In [1]:
from pyspark.sql.types import *
from pyspark.sql import Row
import ast
import json
import pandas as pd
import numpy as np
import requests
import pickle
from datetime import datetime
import matplotlib.pyplot as plt
import os
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
from pyspark.sql import HiveContext
hiveContext=HiveContext(sc)
import sys
from pyspark.sql.functions import lit
from numpy import *
#%matplotlib inline
#dbutils.fs.ls('/mnt/assignment1/csvParquetFiles/')

In [2]:
#df=sqlContext.table("summary")
#df.columns
df.repartition(1).write.format('com.databricks.spark.csv').option("header", "true").save('/mnt/assignment1/allSummary3.csv')

In [3]:
def loadUpdatedTags():
  url="https://s3-us-west-2.amazonaws.com/data-analysis-sachin/tags.json"
  tagsResponse=requests.get(url)
  import json
  j=json.loads(tagsResponse.text)
  dfs=[]
  keys=j.keys()
  for key in keys:
    for d in j[key]['sensorpoints']:
      d['template']=j[key]['template']
      d['name']=j[key]['name']
      d['sensor_id']=str(key)
      df=pd.DataFrame.from_dict(d)
      dfs.append(df)
  print len(keys)
  print len(dfs)
  df=pd.concat(dfs)
  print df.columns
  print len(df)
  return df

tagsdfPd=loadUpdatedTags()
#tagsDF=sqlContext.createDataFrame(tagsdfPd)
#spDF.write.parquet("/mnt/assignment1/csvParquetFiles/tagsDF")

In [4]:
#df=sqlContext.read.parquet("/mnt/assignment1/csvParquetFiles_new/processed_"+str(633))
#print df.count()
#df2=df.select('sensor_id').distinct()
#df2.collect()

In [5]:
#read data from s3
dfs={}
for i in xrange(700,822):
  try:
    path="/mnt/assignment1/csvParquetFiles_new/processed_"+str(i)
    dfs[i]=hiveContext.read.parquet(path)
    print i
    #dfs[i].select('sensor_value').distinct().collect()
  except:
    continue

In [6]:
def getInfo(x,type):
  try:
    if(type=='name'):
      return str(tagsdfPd[tagsdfPd['sensor_id']==x]['name'].tolist()[0])
    elif(type=='template'):
      return str(tagsdfPd[tagsdfPd['sensor_id']==x]['template'].tolist()[0])
  except:
    print x
    return "no room found"

#print getName('876d4cd6-7808-11e3-9b2e-00163e005319')
  

In [7]:
df=dfs[700]
#counts=[dfs[600].count()]
for i in xrange(700,822):
  #counts.append(dfs[i].count())
  df=df.unionAll(dfs[i])

In [8]:
#print sum(counts)
#print df.count()
#print dfs[600].count()
#print dfs[601].count()

In [9]:
#df2=df.rdd
#df2=hiveContext.createDataFrame(df.rdd, samplingRatio=0.2)

In [10]:
#df=hiveContext.sql("select * from allDataBuildingSensorsNew")
#df2=df.withColumn('sensor_value',getName(df.sensor_id))
df2=df.map(lambda row: Row(**dict(row.asDict(), name=getInfo(row.sensor_id,'name'),template=getInfo(row.sensor_id,'template'))))
#df.foreach(getName)
df2=hiveContext.createDataFrame(df2,samplingRatio=0.00001)
print df2.count()

In [11]:
df.write.saveAsTable('data700_822')
#df2.select(['room']).distinct().collect()

In [14]:
#union s3 data to one df
df=dfs[0]
for i in xrange(1,300):
  try:
    df.unionAll(dfs[i])
    print i
  except:
    continue

In [15]:
rooms=list(df.select(['room']).distinct().collect())
rooms=[s.room for s in rooms]
print rooms
#rooms=[u'Rm-4256', u'RM-1145', u'Rm-3252', u'Rm-2148', u'RM-1150', u'Rm-4152', u'CRAC-4', u'RM-1208', u'Rm-3154', u'Rm-3203', u'Rm-2150', u'Rm-2207', u'RM-1212', u'RM-B260', u'AH1-1', u'RM-1106', u'Rm-3213', u'Rm-3215', u'Rm-2214', u'Rm-2219', u'RM-1229', u'Rm-2109', u'Rm-3221', u'Rm-3223', u'CHWP1', u'CHWP3-VFD', u'RM-B215', u'Rm-3111', u'Rm-2221', u'Rm-3114', u'RM-1124', u'RM-1125', u'not_found', u'RM-1126', u'RM-B222', u'Rm-4127', u'Rm-3126', u'Rm-2236', u'Rm-2238', u'RM-B230', u'RM-1139', u'RF2', u'Rm-4254']
#df.count() #=8725890

In [16]:
DF100={}
for i in xrange(251,300):
  try:
    dfRead=dfs[i].drop('sensor_value')
    dfRead=dfRead.dropna()
    roomsList=list(dfRead.select(['room']).distinct().collect())
    roomsList=[s.room for s in roomsList]
    for room in roomsList:
      dfRoom=dfRead.where(dfRead['room']==room)
      if(DF100.has_key(room)):
        DF100[room].unionAll(dfRoom)
      else:
        DF100[room]=dfRoom
  except:
    continue

In [17]:
keys=DF100.keys()
for key in keys:
  try:
    dfRead=DF100[key]
    dfRead=dfRead.map(lambda row: Row(**dict(row.asDict(), timeseries=datetime.strptime(row.timeseries,"%Y-%m-%dT%H:%M:%S+00:00"))))
    dfRead=dfRead.map(lambda row: Row(**dict(row.asDict(), date=datetime.date(row.timeseries))))
    dfRead=dfRead.map(lambda row: Row(**dict(row.asDict(), year=datetime.date(row.timeseries).year)))
    dfRead=dfRead.map(lambda row: Row(**dict(row.asDict(), sensor_value=getSensorValue(row.sensor_id))))
    dfRead=sqlContext.createDataFrame(dfRead)
    dfRead.write.parquet("/mnt/assignment1/csvParquetFiles/room300_"+key)
  except:
    print key

In [18]:
def load(room,year):
  path={}
  path[1]="/mnt/assignment1/csvParquetFiles/room100_"+room
  path[2]="/mnt/assignment1/csvParquetFiles/room200_"+room
  path[3]="/mnt/assignment1/csvParquetFiles/room300_"+room
  path[4]="/mnt/assignment1/csvParquetFiles/room250_"+room
  dfs=[]
  for i in xrange(5):
    try:
      df=sqlContext.read.parquet(path[i])
      df=df.filter(df['year']==year)
      dfs.append(df)
    except:
      print i
  if(len(dfs)>0):
    df=dfs[0]
    for i in xrange(1,len(dfs)):
      df.unionAll(dfs[i])
    return df
  else:
    print "no data for the room"

In [19]:
def dataPlot(dfArgument,timeLowerBound, timUpperBound):
  tempDF=dfArgument[dfArgument.timeseries.between(timeLowerBound, timUpperBound)]
  #if(Rooms!='All'):
  #  tempDF=tempDF[tempDF['room']==Rooms]
  display(tempDF)
  return tempDF

In [20]:
#load room, year
df=load('HW-SYS','2015')
id=df.select('sensor_id').distinct().collect()
print id
#print "tag information for the room is ",tagsDict[id[0].sensor_id]
print df.select('sensor_value').distinct().collect()
print df.count()

In [21]:
dfHW_SYS=dataPlot(df,'2015-03-01 08:00:00', '2015-03-08 17:30:00')

In [22]:
df4126=dataPlot(df,'2015-03-01 08:00:00', '2015-03-08 17:30:00')

In [23]:
# %load PieceWise.py
#Encoder class for spark dataframes
import pandas as pd
from numpy import *
class encoder:
    """
    The encoder/decoder class is the base class for all encoder/decoder pairs.
    Subclasses encode different types of encoding.
    EncoderLearner is a factory class for fitting encoders to data
    """
    def __init__(self,raw):
        """
        given a spark DataFrame or Series (raw), find the best model of a given type
        """
    
    def compress(self):
        """
        given a raw sequence and a model, return a compressed representation.
        """
        self.compressed=None
        return self.compressed
    
    def recon(self,compressed):
        """
        Recreate the original DataFrame or Series, possibly with errors.
        """
        Recon=None
        return Recon
    
    def get_size(self):
        return len(self.compressed)
    
    def compute_error(self,S,compressed=None):
        if type(compressed)==type(None):
            compressed=self.compressed
        R=self.recon(compressed=compressed,index=S.index)
        V=R-S
        V.dropna()
        return sqrt(sum([v*v for v in V.values]))/len(V)

In [24]:
class piecewise_constant(encoder):
    """ 
    Represent the signal using a sequence of piecewise constant functions 
    """
    def __init__(self,raw):
      S=raw
      if type(S) != pd.Series:
        raise 'encode expects pandas Series as input'
      self.index=S.index
      self.Sol=self.fit(S)
    
    # fit uses dynamic programming to find the best piecewise constant solution
    # max_gap is the maximal extent of a single step.
    # Reason for max_gap is that even if the error is small we want to correct
    # it with some minimal frequence. 
    # Not quite a snapshot because the value will not necessarily change after 
    # max_gap is reached.
    def fit(self,S,max_gap=96):
        S[isnan(S)]=0
        _range=np.max(S)-np.min(S)
        # _range is a constant that is added to the error at each stop point
        # Larger values will cause fewer switches.
        print 'range=',_range
        #Dynamic programming
        Sol=[[]]*len(S)  # an array that holds the best partition ending at each point of the sequence.
                # Each element contains a best current value, a pointer to the last change in best 
                # solution so far and the total error of best solution so far.
        for i in range(len(S)):
            if i==0:
                Sol[i]={'prev':None, 'value':S[0], 'error':0.0, 'switch_no':0}
            # Sol is indexed by the location in the sequence S
            # prev: the index of the last switch point
            # value: current prediction value
            # error: cumulative error to this point
            # switch_no: number of switches so far.
            else:
                err0 = Sol[i-1]['error']+(Sol[i-1]['value']-S[i])**2
                best=None
                best_err=1e20
                best_val=S[i]
                for j in xrange(np.max([0,i-max_gap]),i):
                    _mean=np.mean(S[j:i])
                    _std=np.std(S[j:i])
                    err=_std*(i-j)+Sol[j]['error']+_range
                    if err<best_err:
                        best=j
                        best_val=_mean
                        best_err=err
                Sol[i]={'prev':best, 'value':best_val, 'error':best_err,\
                        'switch_no': Sol[best]['switch_no']+1}
            #print '\r',i,Sol[i],
        return Sol
    
    def compress(self,S):
        Switch_points=[]
        i=len(self.Sol)-1                # start from the end 
        while i>0:
            prev=self.Sol[i]['prev']
            value=self.Sol[i]['value']
            if self.Sol[prev]['value'] != value:
                Switch_points.append({'time':S.index[prev],'value':value})
            i=prev
        self.compressed=Switch_points
        return Switch_points

    def recon(self,compressed=None, index=None):
        #print '\nindex=',index==None,'\n'
        #print '\ncompressed=',compressed==None,'\n'
        if(type(index)==type(None)):
            index=self.index
        Recon=pd.Series(index=index)
        
        if(type(compressed)==type(None)):
            compressed=self.compressed
        for e in compressed:
            time=e['time']
            value=e['value']
            Recon[time]=value
            
        return Recon.fillna(method='ffill')

In [25]:
class piecewise_linear(encoder):
    """ 
    Represent the signal using a sequence of piecewise linear functions 
    """
    def __init__(self,raw):
      S=raw
      if type(raw) != pd.Series:
        raise 'encode expects pandas Series as input'
      self.index=raw.index
      self.Sol=self.fit(raw)
    
    # fit uses dynamic programming to find the best piecewise linear solution
    # max_gap is the maximal extent of a single step.
    # Reason for max_gap is that even if the error is small we want to correct
    # it with some minimal frequence. 
    # Not quite a snapshot because the value will not necessarily change after 
    # max_gap is reached.
    def fit(self,S,max_gap=96):
        S[isnan(S)]=0
        _range=np.max(S)-np.min(S)
        # _range is a constant that is added to the error at each stop point
        # Larger values will cause fewer switches.
        print 'range=',_range
        #Dynamic programming
        Sol=[[]]*len(S)  # an array that holds the best partition ending at each point of the sequence.
                # Each element contains a best current value, a pointer to the last change in best 
                # solution so far and the total error of best solution so far.
        for i in range(len(S)):
            if i==0:
                Sol[i]={'prev':None, 'value':S[0], 'error':0.0, 'switch_no':0, 'slope':0}
            # Sol is indexed by the location in the sequence S
            # prev: the index of the last switch point
            # value: current prediction value
            # error: cumulative error to this point
            # switch_no: number of switches so far.
            #slope: slope of th linear line at this point
            else:
                err0 = Sol[i-1]['error']+(Sol[i-1]['value']-S[i])**2
                best=None
                best_err=1e20
                best_val=S[i]
                best_slope=1e20
                for j in xrange(np.max([0,i-max_gap]),i):
                    #_mean=np.mean(S[j:i])
                    _slope=(S[i]-S[j])*1.0/(i-j)
                    #_std=np.std(S[j:i]
                    _val=0
                    _err=0
                    for k in xrange(j,i):
                      _val=Sol[j]['value']+_slope*(k-j)
                      _err+=(Sol[k]['value']-_val)**2
                    err=_err*1.0/(i-j)+Sol[j]['error']+_range
                    _val=Sol[j]['value']+_slope*(i-j)
                    if err<best_err:
                        best=j
                        best_val=_val
                        best_err=err
                        best_slope=_slope
                Sol[i]={'prev':best, 'value':best_val, 'error':best_err,\
                        'switch_no': Sol[best]['switch_no']+1, 'slope':best_slope}
            #print '\r',i,Sol[i],
        return Sol
    
    def compress(self,S):
        Switch_points=[]
        i=len(self.Sol)-1                # start from the end 
        while i>0:
            prev=self.Sol[i]['prev']
            slope=self.Sol[i]['slope']
            value=self.Sol[i]['value']
            if self.Sol[prev]['slope'] != slope:
                Switch_points.append({'time':S.index[prev],'value':value})
            i=prev
        self.compressed=Switch_points
        return Switch_points

    def recon(self,compressed=None, index=None):
        #print '\nindex=',index==None,'\n'
        #print '\ncompressed=',compressed==None,'\n'
        if(type(index)==type(None)):
            index=self.index
        Recon=pd.Series(index=index)
        
        if(type(compressed)==type(None)):
            compressed=self.compressed
        for e in compressed:
            time=e['time']
            value=e['value']
            Recon[time]=value
            
        return Recon.fillna(method='ffill')

In [26]:
def model(A,method):
  #A.count() #=2093
  #A.columns  
  sys.stdout.flush()
  #try:
  pd_df=pd.DataFrame(A.select('timeseries','values').limit(5000).collect(),columns=['timeseries','values'])
  pd_df.plot(kind='line')
  S=pd_df['values']
  _std=np.std(S)
  print "Std dev is ",_std
  if(method=='piecewise_constant'):
    encoder=piecewise_constant(S)
  elif(method=='piecewise_linear'):
    encoder=piecewise_linear(S)
  C=encoder.compress(S)
  compressed_df=pd.DataFrame(C)
  print 'size=',encoder.get_size(),
  error=encoder.compute_error(S,compressed=C)
  print 'error=',error, 'error/_std=',error/_std
  print C
  return compressed_df
  #except:
  #  print "Error!"

In [27]:
#data to be analyzed - dfHW_SYS for time range '2015-03-01 08:00:00', '2015-03-08 17:30:00'

pd_df=pd.DataFrame(dfHW_SYS.select('timeseries','values').limit(5000).collect(),columns=['timeseries','values'])
fig = plt.figure(figsize=(15, 3))
pd_df.plot(kind='line',ax=fig.gca())
display(fig)

In [28]:
compressed_df=model(dfHW_SYS,'piecewise_constant') 
compressed_df=compressed_df.sort(['time'], ascending=[1])
print compressed_df

In [30]:
#piecewise constant analysis for dfHW_SYS for time range '2015-03-01 08:00:00', '2015-03-08 17:30:00'
fig = plt.figure(figsize=(15, 3))
compressed_df.plot(x='time',y='value',kind='line',ax=fig.gca())
display(fig)

In [31]:
compressed_df_linear=model(dfHW_SYS,'piecewise_linear') 
compressed_df_linear=compressed_df_linear.sort(['time'], ascending=[1])
print compressed_df_linear

In [32]:
#piecewise linear for dfHW_SYS for time range '2015-03-01 08:00:00', '2015-03-08 17:30:00'
fig_linear = plt.figure(figsize=(15, 3))
compressed_df_linear.plot(x='time',y='value',kind='line',ax=fig_linear.gca())
print display(fig_linear)

#since this data is mostly step shifted, its better modeled using piecewise constant as against piecewise linear

In [35]:
#Old code below

In [36]:
A=dfHW_SYS
A.count()
A=A.dropna(subset = ['room'])
print 'starting Piecewise, rooms=',rooms
sys.stdout.flush()
cols = list(A.columns)
cols.remove('room')

for room in rooms:
  try:
    #room=rooms[0]
    print '-'*20,room,'-'*20
    sys.stdout.flush()
    DF = A[A['room']==room]
    DF = DF.drop('room')  # remove the entry "Room"
    name='values'
    pd_df=pd.DataFrame(DF.limit(1000).collect(),columns=['TimeStamp','date','sensor_id','sensor_value','values'])
    S=pd_df[name]
    #S=[s.values for s in S][0:200]
    #Spd=pd.Series(S,name='values') 
    _std=np.std(S)
    print 'room=',room,'signal=',name, 'std=',_std,
    encoder=piecewise_constant(S)
    C=encoder.compress()
    print 'size=',encoder.get_size(),
    error=encoder.compute_error(S,compressed=C)
    print 'error=',error, 'error/_std=',error/_std
    print C
  except:
    print room    

In [37]:
display(df5)

In [38]:
df.select('sensor_name').distinct().collect()

In [39]:
display(name_dfs[names2[1]]) #MAIN HW SUPPLY TEMP

In [40]:
display(name_dfs[names2[2]]) #Zone Temperature

In [41]:
display(name_dfs[names2[3]]) #Something

In [42]:
display(name_dfs[names2[4]]) #Warm Cool Adjust

In [43]:
display(name_dfs[names2[5]]) #Al 1 Actual

In [44]:
display(name_dfs[names2[7]]) #Damper Position

In [45]:
display(name_dfs[names2[8]]) #Actual Cooling SetPoint

In [46]:
name_dfs={}
for name in names2:
  name_dfs[name]=dataPlot(df,'2014-03-01 08:00:00', '2014-03-08 17:30:00','All',name)

In [47]:
display(name_dfs[names2[0]]) #Actual heating setpoint

In [48]:
display(name_dfs[names2[1]]) #MAIN HW SUPPLY TEMP

In [49]:
display(name_dfs[names2[2]]) #zone temp

In [50]:
display(name_dfs[names2[3]]) #something

In [51]:
display(name_dfs[names2[6]]) #Supply vel press

In [52]:
display(name_dfs[names2[7]]) #damper 

In [53]:
display(name_dfs[names2[8]]) #Actual cooling setpoint

In [54]:
room_dfs={}
for room in rooms:
  room_dfs[room]=dataPlot(df,'2014-01-01 08:00:00', '2014-01-30 17:30:00',room,'All')

In [55]:
display(room_dfs[rooms[0]])

In [56]:
display(room_dfs[rooms[1]])

In [57]:
display(room_dfs[rooms[2]])

In [58]:
display(room_dfs[rooms[3]])

In [59]:
display(room_dfs[rooms[4]])

In [60]:
display(room_dfs[rooms[5]])

In [61]:
display(room_dfs[rooms[6]])

In [62]:
mayWeek={}
for room in rooms:
  mayWeek[room]=dataPlot(df,'2014-05-06 08:00:00', '2014-05-13 17:30:00',room,'All')

In [63]:
display(mayWeek[rooms[0]])

In [64]:
display(mayWeek[rooms[1]])

In [65]:
display(mayWeek[rooms[3]])

In [66]:
display(mayWeek[rooms[5]])

In [67]:
display(mayWeek[rooms[6]])

In [68]:
#Old code below

In [69]:
#Building sensor:room dictionary
tags=sc.textFile("/mnt/assignment1/tags.json")
tags.cache()
#All sc.textfile reads are single line
tagsDict=ast.literal_eval(tags.take(1)[0])
sensorRoomDict={}
for k in tagsDict.keys():
  ctxts=tagsDict[k]['contexts']
  for c in ctxts:
    if(c['keyword']=='room'):
        sensorRoomDict[str(k)]=str(c['tag'])
        
print sensorRoomDict['876d4cd6-7808-11e3-9b2e-00163e005319']

In [70]:
def jsonParser(line):
  dictRow={}
  row=line.split("}]}}")
  for r in row:
    splits=r.split('"')
    if(len(splits)>=2):
      dictRow[splits[1]]=[splits[3],
    
  return dictRow

In [71]:
#Reading raw_data jsons - simple small file - rawdata_047.json #Using json.loads <-- doesn't ork for most given jsons
rawdata_047 = sc.textFile("/mnt/assignment1/rawdata_047.json")
rawdata_047.cache()
rd047=rawdata_047.take(1)
rd047df=json.loads(str(rd047[0]))
rd047df['33c56e1e-8917-11e2-8a68-00163e005319']['PresentValue']['timeseries']

In [72]:
#from pyspark.sql import SQLContext
#sqlContext = SQLContext
#df_021=sqlContext.read.json("/mnt/assignment1/rawDataJsons/DF_085.json")
#df_044=sc.textFile("/mnt/assignment1/rawDataJsons/DF_085.json", minPartitions = None, use_unicode = True)
#df_021=sc.textFile("/mnt/assignment1/rawDataJsons/DF_021.json", minPartitions = 100)
#df_021_1=sc.textFile("/mnt/assignment1/rawDataJsons/DF_021_split1.json")
df_021_2=sqlContext.read.json("/mnt/assignment1/rawDataJsons/DF_021_split2.json")

In [73]:
df_021_2.head()

In [74]:
#t=df_044.collect()
#df=pd.DataFrame(str(t[0]))
#x=ast.literal_eval(t[0])
#df44=pd.DataFrame(x)
#df44
#t[0][-100:-1]
#t21_1=df_021_1.collect()
t21_1

In [75]:
rawdata_065=sc.textFile("/mnt/assignment1/rawdata/rawdata_065.json")
rawdata_066 = sc.textFile("/mnt/assignment1/rawdata/rawdata_066.json")

In [76]:
tagsDict['e34eb68e-5882-4860-bb40-7832bb7e8a70']

In [77]:
l=rawdata_066.collect()

In [78]:
l2[0].split('"')

In [79]:
splits[0].split('"')

In [80]:
rd066=rawdata_066.map(lambda r:jsonParser(r))

In [81]:
rd066DF=rd066.toDF()
rd066DF.columns

In [82]:
rd010=sqlContext.read.json("/mnt/assignment1/rawdata/rawdata_010.json")
#l009=rd010.repartition(100).collect()

In [83]:
rd065=rawdata_065.map(lambda r:jsonParser(r))

In [84]:
rd065DF=rd065.toDF()
rd065DF.columns #Non existing column in tag

In [85]:
rd065DF.collect()

In [86]:
tagsDict.keys()